### User input

In [9]:
manga_name = input("Enter the name of managa or manwa: ")
manga_name = manga_name.replace(" ", "-")
chapter_range = input("Enter a range (e.g., 1, 100): ")
start, end = map(int, chapter_range.split(','))

In [36]:
# https://www.mangasub.net/manga/solo-leveling/
import requests
from bs4 import BeautifulSoup

chapters = []

for chapter in range(start, end + 1):
    chapter = f"{chapter:03d}"
    url = f'https://www.mangasub.net/manga/{manga_name}/ch-{chapter}/'
    # print(f"URL for Chapter {chapter}: {url}")
    chapters.append(url)

In [42]:
chapter_images = []
for chapter in chapters:
  response = requests.get(chapter, headers=headers)
  soup = BeautifulSoup(response.text, 'html.parser')
  content = soup.find('div', class_="reading-content")
  
  for image in content.findAll('img'):
    chapter_images.append(image['src'].replace("\t\t\t\n\t\t\t", ""))

In [44]:
chapter_images

['https://www.mangasub.net/wp-content/uploads/WP-manga/data/manga_6679c4cc986a3/ch-001/ctc_tog_081_000.jpg',
 'https://www.mangasub.net/wp-content/uploads/WP-manga/data/manga_6679c4cc986a3/ch-001/ctc_tog_081_001.jpg',
 'https://www.mangasub.net/wp-content/uploads/WP-manga/data/manga_6679c4cc986a3/ch-001/ctc_tog_081_002.jpg',
 'https://www.mangasub.net/wp-content/uploads/WP-manga/data/manga_6679c4cc986a3/ch-001/ctc_tog_081_003.jpg',
 'https://www.mangasub.net/wp-content/uploads/WP-manga/data/manga_6679c4cc986a3/ch-001/ctc_tog_081_004.jpg',
 'https://www.mangasub.net/wp-content/uploads/WP-manga/data/manga_6679c4cc986a3/ch-001/ctc_tog_081_005.jpg',
 'https://www.mangasub.net/wp-content/uploads/WP-manga/data/manga_6679c4cc986a3/ch-001/ctc_tog_081_006.jpg',
 'https://www.mangasub.net/wp-content/uploads/WP-manga/data/manga_6679c4cc986a3/ch-001/ctc_tog_081_007.jpg',
 'https://www.mangasub.net/wp-content/uploads/WP-manga/data/manga_6679c4cc986a3/ch-001/ctc_tog_081_008.jpg',
 'https://www.manga

In [71]:
import sqlite3
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

manga_name = input("Enter the name of manga or manhwa: ").lower().replace(" ", "-")
chapter_range = input("Enter a range (e.g., 1, 100): ")
start, end = map(int, chapter_range.split(','))

conn = sqlite3.connect('manga_data.db')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS manga (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    manga_name TEXT,
    chapter TEXT,
    chapter_url TEXT,
    image_url TEXT
    )
''')

# data = []


for chapter in range(start, end + 1):
    chapter_formatted = f"{chapter:03d}"  # Format chapter number with leading zeros
    chapter_url = f'https://www.mangasub.net/manga/{manga_name}/ch-{chapter_formatted}/'
    print(f"Fetching data for Chapter {chapter_formatted} from {chapter_url}")
    
    response = requests.get(chapter_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        content = soup.find('div', class_="reading-content")
        if content:
            image_urls = [img['src'].replace("\t\t\t\n\t\t\t", "") for img in content.find_all('img') if img.has_attr('src') and img['src'].strip()]
            # image_urls_str = ",".join(image_urls)
            image_urls_json = json.dumps(image_urls)
            try:
                cursor.execute('''
                    INSERT INTO manga (manga_name, chapter, chapter_url, image_url)
                    VALUES (?, ?, ?, ?)
                ''', (manga_name.replace("-", " "), chapter_formatted, chapter_url, image_urls_json))  
            except Exception as e:
                print(f"Error inserting data: {e}")
        time.sleep(1)
    else:
        print(f"Failed to fetch Chapter {chapter_formatted}: {response.status_code}")

conn.commit()
conn.close()

print("Data saved to the database successfully!")

Fetching data for Chapter 001 from https://www.mangasub.net/manga/solo-leveling/ch-001/
Fetching data for Chapter 002 from https://www.mangasub.net/manga/solo-leveling/ch-002/
Data saved to the database successfully!
